In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_DC', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.shape

(225427, 64)

In [4]:
df.head()

,cid,createDate,createBy,dataFrom,X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,...,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY,BLOCKKEY,SUBBLOCKKEY,FATALPASSENGER,MAJORINJURIESPASSENGER,MINORINJURIESPASSENGER,UNKNOWNINJURIESPASSENGER
0,1000000001,2020-01-28 05:11:28,David,DC,﻿X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,...,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY,BLOCKKEY,SUBBLOCKKEY,FATALPASSENGER,MAJORINJURIESPASSENGER,MINORINJURIESPASSENGER,UNKNOWNINJURIESPASSENGER
1,1000000002,2020-01-28 05:11:28,David,DC,-77.039863907099999,38.914841272899999,124909612,27031583,16134688,2016-08-12T15:38:49.000Z,...,38.91458,-77.039862,396552.23,138537.33,1cc97d6e7e9b3796ad46cd1c50dd4985,936d5f3d786876148a4686c0025d9712,0,0,0,0
2,1000000003,2020-01-28 05:11:28,David,DC,-76.971960775100001,38.874079514100004,124909613,26650785,11184492,2011-12-17T05:00:00.000Z,...,NaN,NaN,NaN,NaN,b0b71a666c0c754bd03f5c34acbca399,a835014cb0a665be1f69ee603a0bbda1,0,0,0,0
3,1000000004,2020-01-28 05:11:28,David,DC,-76.969975326799997,38.857014623799998,124909614,27069779,16159114,2016-09-19T21:25:08.000Z,...,38.857007,-76.969938,402580.71,132214.86,0eae799caf0901595405a3978a5ad605,a45d468086d6101623bef30c6c704ae6,0,0,0,0
4,1000000005,2020-01-28 05:11:28,David,DC,-77.065500727100002,38.911493645900002,124909615,26507321,11117048,2011-08-12T10:14:00.000Z,...,NaN,NaN,NaN,NaN,4ce0ab057967bdd3153ec55ae35c2863,4ce0ab057967bdd3153ec55ae35c2863,0,0,0,0


In [5]:
df = df[1:]

In [6]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'X', 'Y', 'OBJECTID',
       'CRIMEID', 'CCN', 'REPORTDATE', 'ROUTEID', 'MEASURE', 'OFFSET',
       'STREETSEGID', 'ROADWAYSEGID', 'FROMDATE', 'TODATE', 'MARID', 'ADDRESS',
       'LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD', 'WARD', 'EVENTID',
       'MAR_ADDRESS', 'MAR_SCORE', 'MAJORINJURIES_BICYCLIST',
       'MINORINJURIES_BICYCLIST', 'UNKNOWNINJURIES_BICYCLIST',
       'FATAL_BICYCLIST', 'MAJORINJURIES_DRIVER', 'MINORINJURIES_DRIVER',
       'UNKNOWNINJURIES_DRIVER', 'FATAL_DRIVER', 'MAJORINJURIES_PEDESTRIAN',
       'MINORINJURIES_PEDESTRIAN', 'UNKNOWNINJURIES_PEDESTRIAN',
       'FATAL_PEDESTRIAN', 'TOTAL_VEHICLES', 'TOTAL_BICYCLES',
       'TOTAL_PEDESTRIANS', 'PEDESTRIANSIMPAIRED', 'BICYCLISTSIMPAIRED',
       'DRIVERSIMPAIRED', 'TOTAL_TAXIS', 'TOTAL_GOVERNMENT',
       'SPEEDING_INVOLVED', 'NEARESTINTROUTEID', 'NEARESTINTSTREETNAME',
       'OFFINTERSECTION', 'INTAPPROACHDIRECTION', 'LOCATIONERROR',
       'LASTUPDATEDATE', '

In [7]:
df2 = pd.DataFrame()
df2['cid'] = df.cid
df2['city'] = df.dataFrom
df2['crash_time'] = df.REPORTDATE
df2['latitude'] = df.LATITUDE
df2['longitude'] = df.LONGITUDE
df2['street_name'] = df.NEARESTINTSTREETNAME

df2['crash_time'] = pd.to_datetime(df2.crash_time)
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [8]:
injured_col_type = ['MAJORINJURIES','MINORINJURIES','UNKNOWNINJURIES','FATAL']
injured_col=[]
df.loc[:,injured_col] = df.loc[:,injured_col].astype('int')
for t in injured_col_type:
    injured_col.append([col for col in df.columns if t in col])
injured_col = sum(injured_col, [])
df2['injured_count'] = df.loc[:,injured_col].sum(axis=1)

In [9]:
df2.head()

,cid,city,crash_time,latitude,longitude,street_name,crash_date,injured_count
1,1000000002,DC,2016-08-12 15:38:49+00:00,38.91458,-77.039862,Alley-47071322,2016-08-12,0.0
2,1000000003,DC,2011-12-17 05:00:00+00:00,38.874461,-76.971676,Ramp-36001712,2011-12-17,0.0
3,1000000004,DC,2016-09-19 21:25:08+00:00,38.857007,-76.969938,Driveway-58011932,2016-09-19,0.0
4,1000000005,DC,2011-08-12 10:14:00+00:00,38.911486,-77.065498,Q ST NW,2011-08-12,0.0
5,1000000006,DC,2017-05-17 18:35:40+00:00,38.822068,-77.016679,Not Available,2017-05-17,0.0


In [10]:
df2.to_sql('cleaned_crash_DC',con = db_connection, if_exists="replace",index=False)

In [11]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)